**This notebook infers vertebae classification based on EfficientNet vertebrae classification model and format into the structure to feed into the EfficientNet fracture prediction model**

## Load Packages

In [1]:
#! pip install pylibjpeg pylibjpeg-libjpeg pydicom
#! pip install -U python-gdcm
#! pip install opencv-python-headless
#! pip install nibabel
#! pip install tensorflow
#! pip install --upgrade tensorflow_hub
#! pip install --upgrade tensorflow-gpu
#! pip install pydicom
#! pip install path.py

In [1]:
import sys
sys.path.append('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\kerasapplications')
sys.path.append('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\efficientnet-keras-source-code\\')
import keras_applications
import efficientnet.tfkeras as efficientnet

C:\Users\public.DESKTOP-P7554KD\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import cv2
import os
from os import listdir
import re
import gc
from time import time
from datetime import datetime

import gdcm
import pydicom as dicom
from pydicom import dcmread
import pylibjpeg
from pydicom.pixel_data_handlers.util import apply_voi_lut
from pydicom.data import get_testdata_files

from path import Path
import scipy.ndimage
from tqdm import tqdm
import nibabel as nib
from pprint import pprint
from time import time
import itertools
from skimage import measure 
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from glob import glob
import warnings
import dask.array as da

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, cohen_kappa_score, confusion_matrix
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import losses, callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import tensorflow_hub as hub
from keras.preprocessing.image import ImageDataGenerator
from keras import layers

from PIL import Image as im
import traceback

import imageio


#from efficientnet_v2 import EfficientNetV2S
%matplotlib inline
sns.set(style='darkgrid', font_scale=1.6)

## Load Training Scans

In [5]:
#Loading Scans
def atoi(text):
    return int(text) if text.isdigit() else text
def natural_keys(text):
    return [atoi(c) for c in re.split(r'(\d+)', text)]

# Load the scans in given folder path
def load_scan(path):
    
    dcm_paths = glob(f"{path}/*")
    dcm_paths.sort(key=natural_keys)
    
    patient_scan = [dicom.dcmread(paths) for paths in dcm_paths]
    
    return patient_scan

## HU Rescale

In [6]:
def get_pixels_hu(slices):
   
    image = np.stack([cv2.resize(s.pixel_array,(256,256),interpolation = cv2.INTER_NEAREST) for s in slices])
    #image = np.stack([cv2.cvtColor(s.pixel_array.reshape(512,512),cv2.COLOR_GRAY2RGB) for s in slices])

    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16) #int16
    image = da.from_array(image) #Using Dask to speed up processing
    
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
    image[image <= -1000] = 0
    
    # Convert to Hounsfield units (HU)
        
    intercept = da.from_array([slices[slice_number].RescaleIntercept for slice_number in range(len(slices))])
    slope = da.from_array([slices[slice_number].RescaleSlope for slice_number in range(len(slices))])
    
    intercept=intercept.reshape((-1,1,1))
    slope=slope.reshape((-1,1,1))
    #print(slope.shape)
    #print(image.shape)
    image= slope * image.astype("float64")
    #print(image.shape)   
    image+= intercept
    #print(image.shape)      
    return image.astype("int16")

## Normalization

In [7]:
MIN_BOUND = 150.0
MAX_BOUND = 2050.0
    
def normalize(image):
    image = (image - MIN_BOUND)*255.0 / (MAX_BOUND - MIN_BOUND)
    image[image>255] = 255.
    image[image<0] = 255.
    
    image = image.astype(np.int16)
    return image

## Inference

In [8]:
#Getting patient with mask
dicom_paths = glob("C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\train_images\\*")
all_patient=[]
for path in dicom_paths:
    all_patient.append((path.rsplit("\\",1)[-1]))#Patient with mask present

In [35]:
len(all_patient)

2019

In [27]:
len(infer_patient)

2019

In [39]:
all_patient[4]

'1.2.826.0.1.3680043.10032'

In [9]:
def infer():
    
    infer_ds_x=[]
    patient_id = []
    image_id = []
    #for i in range(start,start+10):
    for i in range(len(all_patient[:300])):
        patient_ID=all_patient[i]
        print(patient_ID)
        patient_scan=load_scan(f"C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\train_images\\{patient_ID}") #need to change back
        patient_hu=get_pixels_hu(patient_scan)
        patient_hu_normalised=normalize(patient_hu)
        
        for j in tqdm(range(0,len(patient_scan))):
           
            infer_ds_x.append(patient_hu_normalised[j].astype(np.uint8))
        
        dcm_paths = glob(f"C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\train_images\\{patient_ID}\\*")
        dcm_paths.sort(key=natural_keys)
    
        for paths in dcm_paths:
            patient_id.append(paths.split('\\')[-2])
            image_id.append(paths.split('\\')[-1][:-4])
            
    return ({'patient_id': patient_id, 'image_id': image_id}, infer_ds_x)

In [10]:
infer_ds_x_updated=infer()[1]

1.2.826.0.1.3680043.10001


100%|██████████| 268/268 [00:00<00:00, 1402.81it/s]


1.2.826.0.1.3680043.10005


100%|██████████| 259/259 [00:00<00:00, 1729.66it/s]


1.2.826.0.1.3680043.10014


...

100%|██████████| 275/275 [00:00<00:00, 1955.72it/s]


1.2.826.0.1.3680043.14654


100%|██████████| 383/383 [00:00<00:00, 1885.49it/s]


1.2.826.0.1.3680043.14669


100%|██████████| 267/267 [00:00<00:00, 1898.71it/s]


In [22]:
len(infer_ds_x_updated)

106400

In [ ]:
infer()[0]

In [12]:
infer_ds_x_updated_np=np.array(infer_ds_x_updated)

In [13]:
infer_ds_x_updated_np=np.reshape(infer_ds_x_updated_np,(-1,512,512,1)) #(-1,512,512,1)

In [14]:
model = keras.models.load_model('C:\\Users\\public.DESKTOP-P7554KD\\Downloads\\Classifier_v3.h5')

In [15]:
infer_pred = model.predict(infer_ds_x_updated_np, use_multiprocessing=True) 

832/832 [==============================] - 15430s 19s/step


In [16]:
np.savetxt("infer_pred.txt", infer_pred)
#infer_pred.to_csv('infer_pred.csv', index=False)

In [18]:
infer_pred

array([[0.69692314, 0.37821338, 0.27100116, ..., 0.36937758, 0.41666928,
        0.42394972],
       [0.73315144, 0.34157723, 0.18615925, ..., 0.32516384, 0.42347708,
        0.44838518],
       [0.65651876, 0.4009105 , 0.22304966, ..., 0.40478536, 0.45719445,
        0.41393763],
       ...,
       [0.83746016, 0.13958631, 0.11340163, ..., 0.32859543, 0.5526645 ,
        0.81542367],
       [0.89727604, 0.11709262, 0.09312554, ..., 0.24854207, 0.45906565,
        0.85071135],
       [0.8766982 , 0.12272485, 0.10013004, ..., 0.22116975, 0.4613741 ,
        0.8464324 ]], dtype=float32)

In [17]:
len(infer_ds_x_updated_np)

26600

In [19]:
len(infer_pred)

26600

In [24]:
c1 = []
c2 = []
c3 = []
c4 = []
c5 = []
c6 = []
c7 = []

for i in infer_pred:
    for j in range(4):
        c1.append(i[1])
        c2.append(i[2])
        c3.append(i[3])
        c4.append(i[4])
        c5.append(i[5])
        c6.append(i[6])
        c7.append(i[7])

In [23]:
infer_label = pd.read_csv('infer_label.csv')
infer_label

,patient_id,image_id
0,1.2.826.0.1.3680043.10001,1
1,1.2.826.0.1.3680043.10001,2
2,1.2.826.0.1.3680043.10001,3
3,1.2.826.0.1.3680043.10001,4
4,1.2.826.0.1.3680043.10001,5
...,...,...
106395,1.2.826.0.1.3680043.14669,263
106396,1.2.826.0.1.3680043.14669,264
106397,1.2.826.0.1.3680043.14669,265
106398,1.2.826.0.1.3680043.14669,266


In [26]:
vertebrae_infer = infer_label
vertebrae_infer['C1'] = c1
vertebrae_infer['C2'] = c2
vertebrae_infer['C3'] = c3
vertebrae_infer['C4'] = c4
vertebrae_infer['C5'] = c5
vertebrae_infer['C6'] = c6
vertebrae_infer['C7'] = c7
vertebrae_infer

,patient_id,image_id,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.10001,1,0.378213,0.271001,0.257165,0.321376,0.270718,0.369378,0.416669
1,1.2.826.0.1.3680043.10001,2,0.378213,0.271001,0.257165,0.321376,0.270718,0.369378,0.416669
2,1.2.826.0.1.3680043.10001,3,0.378213,0.271001,0.257165,0.321376,0.270718,0.369378,0.416669
3,1.2.826.0.1.3680043.10001,4,0.378213,0.271001,0.257165,0.321376,0.270718,0.369378,0.416669
4,1.2.826.0.1.3680043.10001,5,0.341577,0.186159,0.190069,0.276143,0.230544,0.325164,0.423477
...,...,...,...,...,...,...,...,...,...
106395,1.2.826.0.1.3680043.14669,263,0.204498,0.151299,0.131888,0.199384,0.195229,0.223207,0.441043
106396,1.2.826.0.1.3680043.14669,264,0.179163,0.154142,0.103150,0.200212,0.167246,0.198423,0.438631
106397,1.2.826.0.1.3680043.14669,265,0.179163,0.154142,0.103150,0.200212,0.167246,0.198423,0.438631
106398,1.2.826.0.1.3680043.14669,266,0.179163,0.154142,0.103150,0.200212,0.167246,0.198423,0.438631


In [28]:
vertebrae_infer.to_csv('class_pred.csv', index=False)